In [1]:
import pandas as pd
import urllib.request
import numpy as np
import math


## How to process real world PDB files

Be careful with:
* For the same Uniprot ID there may be tens of PDB files
* It is important to select the longest AA chains with coordinates - other criteria might be used
* sometimes several chains, with different parts of the sequence represented
* The Uniprot format for PDB entries is inconsistent and the code below has been wicked for the 144 proteins in our dataset. It is not guaranteed that it will work for other PDB files


In [2]:
# Helper function to download data

def get_uniprot_PDB_ids(uniprot_list):
    D = {}

    for c, prot in enumerate(uniprot_list):
        url="https://rest.uniprot.org/uniprotkb/{}.txt".format(prot)
        contents = urllib.request.urlopen(url).read()
        uniprot_res=contents.decode("utf-8").split("\n")
        best_siz = -1
        best_pdb = ""
        best_chn = []
        print(c, "-->", prot)
        for lin in uniprot_res:
            if 'DR   PDB;' in lin:
                pos_eq=lin.find("=")
                pos_pdb = lin.find("PDB;")+4
                pos_pdb_end = lin[pos_pdb:].find(";")
                pdb_file=lin[pos_pdb:pos_pdb+pos_pdb_end]
                if lin[pos_eq:].find(",")==-1:
                    #regular: - nothing after
                    pos_A=lin.rfind(";")+2
                    rng=lin[pos_eq+1:]
                    strt, end = [i for i in rng[:-1].split("-")]
                    if len(strt)>0 and len(end)>0:
                        strt, end=abs(int(strt)), abs(int(end))
                        siz = end - strt + 1
                        chains=[(lin[pos_A], strt, end)]
                        #print(rng, type(strt), type(end), siz)
                    else:
                        siz=-1
                else:
                    pos_A=lin.rfind(";")+2 #last ;
                    S    = lin[pos_A:]
                    seqs = S.split(",")
                    siz=0
                    chains=[]
                    for s in seqs:
                        s=s.strip().replace(".", "")
                        pos_eq=s.find("=")
                        rng = s[pos_eq+1:]
                        strt, end = [abs(int(i)) for i in rng.split("-")]
                        siz += end - strt + 1
                        #print(rng, strt, end, siz)
                        chains.append((s[0], strt, end))
                        
                if siz > best_siz:
                    best_siz = siz
                    best_pdb = pdb_file
                    best_chn = chains
                    #print(prot, best_pdb, best_siz)
                        
        D[prot]=(best_pdb.strip(), best_siz, best_chn)
    return D
                
                    
                    


Here's a test

In [3]:
up_ids=[lin.strip() for lin in open("ids_144.txt", "rt").readlines()]
d = get_uniprot_PDB_ids(up_ids[20:21])
d

0 --> P21452


{'P21452': ('7XWO', 345, [('B', 1, 345)])}

## Identify the best PDB files for each Uniprot ID

It will produce a list with Uniprot_id, PDB Id, n. of amino acids with coords, and chains to use

In [4]:
import pickle

up_ids=[lin.strip() for lin in open("ids_144.txt", "rt").readlines()]
data = get_uniprot_PDB_ids(up_ids)

up_pdbs=[(ui, data[ui][0], data[ui][1], data[ui][2]) for ui in up_ids]
pickle.dump(up_pdbs, open("uniprot_pdbs.pickle", "wb"))

0 --> O14842
1 --> O43193
2 --> O43613
3 --> O43614
4 --> O95665
5 --> P05067
6 --> P07550
7 --> P08172
8 --> P08173
9 --> P08588
10 --> P08908
11 --> P08912
12 --> P08913
13 --> P0DMS8
14 --> P11229
15 --> P13945
16 --> P14416
17 --> P18089
18 --> P18825
19 --> P20309
20 --> P21452
21 --> P21453
22 --> P21554
23 --> P21728
24 --> P21730
25 --> P21731
26 --> P21917
27 --> P21918
28 --> P24530
29 --> P25021
30 --> P25024
31 --> P25025
32 --> P25098
33 --> P25100
34 --> P25101
35 --> P25103
36 --> P25106
37 --> P25116
38 --> P25929
39 --> P28221
40 --> P28222
41 --> P28223
42 --> P28335
43 --> P28336
44 --> P28566
45 --> P28845
46 --> P29274
47 --> P29275
48 --> P29371
49 --> P30411
50 --> P30518
51 --> P30542
52 --> P30550
53 --> P30556
54 --> P30559
55 --> P30872
56 --> P30874
57 --> P30939
58 --> P30968
59 --> P30989
60 --> P31391
61 --> P32238
62 --> P32239
63 --> P32245
64 --> P32246
65 --> P32745
66 --> P33032
67 --> P34947
68 --> P34969
69 --> P34972
70 --> P34995
71 --> P34998
72

In [ ]:
data

## Actually retrieve the PDB files and process them 

* The code below is mostly without changes from the Alphafold approach
* N. BITS = 16381 (closest prime below 2^14)
* it will produce for each PDB file the respective fingerprints

In [5]:
import requests

aas3to1 = {'ALA': 'A',
        'ARG': 'R',
        'ASN': 'N',
        'ASP': 'D',
        'ASX': 'B',
        'CYS': 'C',
        'GLU': 'E',
        'GLN': 'Q',
        'GLX': 'Z',
        'GLY': 'G',
        'HIS': 'H',
        'ILE': 'I',
        'LEU': 'L',
        'LYS': 'K',
        'MET': 'M',
        'PHE': 'F',
        'PRO': 'P',
        'SER': 'S',
        'THR': 'T',
        'TRP': 'W',
        'TYR': 'Y',
        'VAL': 'V'}

import scipy
def extractAtomData(data, chains):

    #creades distance matrix, atom coordinates and atoms -> amino acids
    atoms_aas=[]
    coords=[]
    for line in data:
        #fs=line.strip().split()
        ATOM = line[:4]
        if ATOM=="ATOM":
            #ATOM, aid, atm, aa, lixo, aaid, x,y,z, s1, s2, atm2=fs
            CHAIN =line[21]
            if CHAIN in chains:
                aid = line[6:11].strip()
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                aa = line[17:20].strip()
                aaid = line[22:26].strip()
                aid=int(aid)
                #x,y,z, = float(x), float(y), float(z)
                atoms_aas.append(aas3to1[aa]+aaid)
                coords.append([x,y,z])
    
    coords=np.array(coords)
    atoms_aas=np.array(atoms_aas)        
    DM = scipy.spatial.distance_matrix(coords, coords)
    return DM, coords, atoms_aas



def process_fp(fp):
    #returns processed aminoacids without ids from aas, but distinguishing different occurrences of the same AA
    L=sorted(list(fp))
    idx=0
    prev=L[0][0]
    L2=[prev+str(idx)]
    for i in L[1:]: 
        if i[0]==prev: idx+=1
        else: idx=0
        prev=i[0]
        L2.append(prev+str(idx))
    return frozenset(L2)

def compute_fps(DM, coords, atoms_aas, max_dist=5):
    fps=[]
    N=coords.shape[0]
    for i in range(N) :
        fp=set(atoms_aas[DM[i]<max_dist])
        if len(fp)>2:  
            fps.append(process_fp(fp))
    return set(fps)

def getFPs(pdb_id, chains, max_dist=5):
    pdb_file_url = f"https://www.ebi.ac.uk/pdbe/entry-files/download/pdb{pdb_id.lower()}.ent"
    pdb_file_response = requests.get(pdb_file_url)
    pdb_file_response.raise_for_status()  # Raise an exception for bad responses
    data = pdb_file_response.content.decode("utf-8").split("\n")    
    dm, crds, aaas = extractAtomData(data, chains)
    fps=compute_fps(dm, crds, aaas, max_dist)
    
    return fps

def h(fp_set, n=16381):
    fp = [0]*n
    hs = [hash(h) % n for h in fp_set]
    hs = set(hs)
    for item in hs:
        fp[item] = 1
    return fp



In [6]:

dist = 5
fails=0
for i, (up, pdb, siz, chains) in enumerate(up_pdbs):
    the_chains=[c for c, st, end in chains]
    print(i, up, pdb, siz, the_chains, end=" ")
    if siz > -1:
        fps= getFPs(pdb, the_chains, dist)
        fps = h(fps)
        pickle.dump(fps, open("../../Desktop/PDB_data/fp_dist{}_ki/{}_D{}.pickle".format(dist,up,dist), "wb"))
        print("DONE!")
    else:
        print("not present...")
        fails+=1
print("N. Fails:", fails)

0 O14842 8EIT 299 ['R'] DONE!
1 O43193 8IBU 412 ['R'] DONE!
2 O43613 6TQ9 354 ['A'] DONE!
3 O43614 7SQO 444 ['R'] DONE!
4 O95665  -1 [] not present...
5 P05067 5BUO 341 ['A'] DONE!
6 P07550 2R4R 365 ['A'] DONE!
7 P08172 4MQS 466 ['A'] DONE!
8 P08173 7TRK 479 ['R'] DONE!
9 P08588 7BTS 346 ['A'] DONE!
10 P08908 7E2X 398 ['R'] DONE!
11 P08912 6OL9 306 ['A', 'A'] DONE!
12 P08913 7EJ0 465 ['R'] DONE!
13 P0DMS8  -1 [] not present...
14 P11229 6OIJ 459 ['R'] DONE!
15 P13945  -1 [] not present...
16 P14416 7JVR 443 ['R'] DONE!
17 P18089 6K42 444 ['R'] DONE!
18 P18825 6KUW 300 ['A', 'A'] DONE!
19 P20309 8E9W 545 ['A'] DONE!
20 P21452 7XWO 345 ['B'] DONE!
21 P21453 7EO4 382 ['A'] DONE!
22 P21554 6N4B 472 ['R'] DONE!
23 P21728 7CKW 446 ['R'] DONE!
24 P21730 8HK5 350 ['A'] DONE!
25 P21731  -1 [] not present...
26 P21917 5WIU 313 ['A', 'A'] DONE!
27 P21918 8IRV 477 ['R'] DONE!
28 P24530 8HBD 398 ['R'] DONE!
29 P25021 7UL3 311 ['A', 'A'] DONE!
30 P25024 8IC0 350 ['A'] DONE!
31 P25025 6LFM 360 ['R'] 

In [7]:
113/144

0.7847222222222222

## Create the FP matrix for PCA

Finally we can process the PCA of the fingerprint matrix

In [8]:
dist=5 #still

#This is really important! The IDs of the proteins for which we can make models
prots_ok = [up for up, pdb, siz, chains in up_pdbs if siz>0]

N=len(prots_ok)
M=16381 #fp num bits
FP_mat=np.zeros((N,M), int)
for i,prot in enumerate(prots_ok):
    FP_mat[i,:]=pickle.load(open("../../Desktop/PDB_data/pdb_paas/{}_D{}.pickle".format(prot,dist), "rb"))


FP_mat[:10, :20]


array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=94)    # <- 90% variance 
pca.fit(FP_mat)
#for i, evr in enumerate(pca.explained_variance_ratio_):
#    print(i, evr, pca.explained_variance_ratio_[:i+1].sum())
FP_mat_pc=pca.transform(FP_mat)
FP_mat_pc.shape

(113, 94)

In [10]:
with np.printoptions(precision=4, suppress=True):
    print(FP_mat_pc[:30, :6])

[[ -3.5322  -0.467    0.8442   0.0368   0.9345  -0.6042]
 [ -2.9772  -2.1128   0.8218  -0.3365  -0.0514  -0.8317]
 [ -1.5327  -1.3778   0.6227   0.4348   0.5449  -0.8712]
 [  7.0782   2.0974  -2.5411   0.8904   1.172    6.1075]
 [ -6.1487  -1.8197   0.6819   0.3906  -0.0641  -1.5571]
 [ -5.4575  -0.5417   1.7154  -0.5141   0.0476  -0.5196]
 [ -2.9708  -0.5434   1.2488  -0.3926  -0.8396   0.0683]
 [ -2.5091  -0.8073   0.2025  -1.0201   0.6703  -0.1879]
 [  5.2397   7.8242 -23.6397 -11.4635  -1.0543   0.402 ]
 [ -3.1936  -1.2315   1.2402  -1.207   -0.1361   0.1872]
 [  3.3464   3.4332 -16.4992  -8.9099  -1.8966   1.7292]
 [ -3.4219  -1.3685   1.5388  -0.5497  -0.3422  -0.556 ]
 [ -2.1733  -0.29     0.0061  -1.4945  -0.5369  -0.0661]
 [ -2.8394  -1.4331   1.7161  -1.2895   0.1176  -0.8376]
 [ -2.7725  -0.9872   1.2544   0.2339  -0.6769   0.1034]
 [  5.5303   1.6524   0.2447   5.3179   5.5337   0.8362]
 [ -2.5769  -0.9288  -0.0819  -0.6815   1.1657  -0.0905]
 [ -3.1831  -1.4232   1.0379   

## Waiting for the molecular data

This matrix can now finally be processed with the molecular information for generating the data sets for the models

First get the UP_Ids for the models

In [26]:
pid='O43193'
mol_data = pickle.load(open('../../Desktop/PDB_data/mol_data/{}_with_id.pickle'.format(pid),'rb'))
N = len(mol_data)
M = len(mol_data[0][1])
N,M

(97, 2048)

In [37]:
X_list=[]
y_list=[]
for i, pid in enumerate(prots_ok):
    mol_data = pickle.load(open('../../Desktop/PDB_data/mol_data/{}_with_id.pickle'.format(pid),'rb'))
    print(i, pid, len(mol_data))
    for mid, fp, y in mol_data:
        X_list.append(fp + list(FP_mat_pc[i])) 
        y_list.append(y)

0 O14842 139
1 O43193 97
2 O43613 1574
3 O43614 1732
4 P05067 382
5 P07550 630
6 P08172 1565
7 P08173 853
8 P08588 429
9 P08908 4741
10 P08912 780
11 P08913 914
12 P11229 1452
13 P14416 8045
14 P18089 558
15 P18825 683
16 P20309 1525
17 P21452 862
18 P21453 139
19 P21554 3991
20 P21728 1401
21 P21730 64
22 P21917 2630
23 P21918 526
24 P24530 181
25 P25021 533
26 P25024 102
27 P25025 218
28 P25098 98
29 P25101 236
30 P25103 920
31 P25106 94
32 P25116 78
33 P25929 654
34 P28221 1085
35 P28222 1040
36 P28223 4219
37 P28335 2633
38 P28566 207
39 P28845 321
40 P29274 5578
41 P29275 2652
42 P30411 321
43 P30518 439
44 P30542 4812
45 P30550 44
46 P30556 149
47 P30559 701
48 P30874 440
49 P30939 137
50 P30968 734
51 P30989 190
52 P31391 316
53 P32238 266
54 P32239 522
55 P32245 1946
56 P32246 160
57 P34947 368
58 P34969 2577
59 P34972 4717
60 P34998 1506
61 P35367 1526
62 P35372 4629
63 P35408 544
64 P35414 230
65 P35462 5106
66 P37288 787
67 P41143 4186
68 P41145 3907
69 P41146 1239
70 P41594

## Making models

First partition the dataset into training and testing

In [48]:
from sklearn.model_selection import train_test_split

X = np.array(X_list)
y = np.array(y_list)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)
X_train.shape, X_test.shape

((97592, 2142), (24398, 2142))

Fit the sucker - **THIS TAKES TIME 1-2 HOURS**

In [44]:
from sklearn.ensemble import RandomForestRegressor

mdl=RandomForestRegressor(n_estimators=200, n_jobs=6)
mdl.fit(X_train, y_train)


RandomForestRegressor(n_estimators=200, n_jobs=6)

In [47]:
from sklearn.metrics import mean_squared_error, explained_variance_score

preds = mdl.predict(X_test)
rmse_test = mean_squared_error(y_test, preds, squared=False) 
expvar_test = explained_variance_score(y_test, preds)

rmse_test, expvar_test

(0.17016729004459794, 0.7239023891825048)

## Now with Alphafold


In [51]:
dist = 5
N=len(prots_ok)
M=16381 #fp num bits
FP_mat_AF=np.zeros((N,M), int)
for i, prot in enumerate(prots_ok):
    FP_mat_AF[i,:]=pickle.load(open("../../Desktop/PDB_data/af_paas/{}_D{}.pickle".format(prot,dist), "rb"))

print(FP_mat_AF.shape)
FP_mat_AF[:10, :20]

(113, 16381)


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Same drill - must do the PCA

In [53]:
pca = PCA(n_components=96)    # <- 90% variance 
pca.fit(FP_mat_AF)
#for i, evr in enumerate(pca.explained_variance_ratio_):
#    print(i, evr, pca.explained_variance_ratio_[:i+1].sum())
FP_mat_AF_pc=pca.transform(FP_mat_AF)
FP_mat_AF_pc.shape

(113, 96)

Make the dataset with the same procedure

In [54]:
X_AF_list=[]
y_AF_list=[]
for i, pid in enumerate(prots_ok):
    mol_data = pickle.load(open('../../Desktop/PDB_data/mol_data/{}_with_id.pickle'.format(pid),'rb'))
    print(i, pid, len(mol_data))
    for mid, fp, y in mol_data:
        X_AF_list.append(fp + list(FP_mat_AF_pc[i])) 
        y_AF_list.append(y)

0 O14842 139
1 O43193 97
2 O43613 1574
3 O43614 1732
4 P05067 382
5 P07550 630
6 P08172 1565
7 P08173 853
8 P08588 429
9 P08908 4741
10 P08912 780
11 P08913 914
12 P11229 1452
13 P14416 8045
14 P18089 558
15 P18825 683
16 P20309 1525
17 P21452 862
18 P21453 139
19 P21554 3991
20 P21728 1401
21 P21730 64
22 P21917 2630
23 P21918 526
24 P24530 181
25 P25021 533
26 P25024 102
27 P25025 218
28 P25098 98
29 P25101 236
30 P25103 920
31 P25106 94
32 P25116 78
33 P25929 654
34 P28221 1085
35 P28222 1040
36 P28223 4219
37 P28335 2633
38 P28566 207
39 P28845 321
40 P29274 5578
41 P29275 2652
42 P30411 321
43 P30518 439
44 P30542 4812
45 P30550 44
46 P30556 149
47 P30559 701
48 P30874 440
49 P30939 137
50 P30968 734
51 P30989 190
52 P31391 316
53 P32238 266
54 P32239 522
55 P32245 1946
56 P32246 160
57 P34947 368
58 P34969 2577
59 P34972 4717
60 P34998 1506
61 P35367 1526
62 P35372 4629
63 P35408 544
64 P35414 230
65 P35462 5106
66 P37288 787
67 P41143 4186
68 P41145 3907
69 P41146 1239
70 P41594

In [55]:
Xaf = np.array(X_AF_list)
yaf = np.array(y_AF_list)
Xaf_train, Xaf_test, yaf_train, yaf_test= train_test_split(Xaf, yaf, test_size=0.2, random_state=123)
Xaf_train.shape, Xaf_test.shape

((97592, 2144), (24398, 2144))

In [56]:
y_test[:20]

array([0.93   , 0.935  , 0.     , 0.6925 , 0.6875 , 0.465  , 0.295  ,
       0.95   , 0.38   , 0.2575 , 0.4925 , 0.90375, 0.     , 0.4925 ,
       0.73   , 0.615  , 0.7225 , 0.     , 0.275  , 0.     ])

In [57]:
yaf_test[:20]

array([0.93   , 0.935  , 0.     , 0.6925 , 0.6875 , 0.465  , 0.295  ,
       0.95   , 0.38   , 0.2575 , 0.4925 , 0.90375, 0.     , 0.4925 ,
       0.73   , 0.615  , 0.7225 , 0.     , 0.275  , 0.     ])

Train the model...

In [58]:
mdl=RandomForestRegressor(n_estimators=200, n_jobs=6)
mdl.fit(Xaf_train, yaf_train)

RandomForestRegressor(n_estimators=200, n_jobs=6)

In [59]:
preds = mdl.predict(Xaf_test)
rmse_test = mean_squared_error(yaf_test, preds, squared=False) 
expvar_test = explained_variance_score(yaf_test, preds)

rmse_test, expvar_test

(0.16985596544941534, 0.7249091224664415)